In [1]:
from utils.imports import *
from utils.functions import *
from utils.paths import *

%matplotlib inline

from sklearn.cluster import KMeans
dropout_rate = 0.35
width = 32

Using TensorFlow backend.


In [2]:
PATH = {'src' : full_lung_train,
       'annotations_path' : annotations_train_path,}

In [49]:
patients = get_dirfiles(PATH['src'])
img = np.load('/Volumes/solo/ali/Data/train_mask/train_full_lung/LKDS-00001.npy')
df = pd.read_csv(PATH['annotations_path'] + 'annotations_final2cls.csv')
df2 = pd.read_csv(PATH['annotations_path'] + 'annotations.csv')
pred = pd.read_csv(PATH['annotations_path'] + 'annotations_pred.csv')

In [47]:
df

,seriesuid,coordX,coordY,coordZ,diameter_mm,origin1,origin2,origin3,new_spacing1,new_spacing2,new_spacing3,offset,voxel_x,voxel_y,voxel_z,intersect,indiam,intersect64
0,LKDS-00001,60.000119,-3.999910,51.501425,6.873059,49.50,-206.500,-207.500,1.000712,1.000000,1.000000,50,2.000000,252.500000,317.500000,0.0,0.0,0.0
1,LKDS-00001,-83.999945,47.000112,117.548433,6.873059,49.50,-206.500,-207.500,1.000712,1.000000,1.000000,50,68.000000,303.500000,173.500000,0.0,0.0,0.0
2,LKDS-00001,-73.499941,69.500122,130.557692,6.203505,49.50,-206.500,-207.500,1.000712,1.000000,1.000000,50,81.000000,326.000000,184.000000,0.0,0.0,0.0
3,LKDS-00001,-76.928983,-50.645135,230.202530,17.981062,49.50,-206.500,-207.500,1.000712,1.000000,1.000000,50,180.573916,205.854796,180.570959,1.0,1.0,1.0
4,LKDS-00001,118.821208,37.080885,272.085827,9.774699,49.50,-206.500,-207.500,1.000712,1.000000,1.000000,50,222.427403,293.580777,376.321063,0.0,0.0,0.0
5,LKDS-00001,42.928682,79.071555,272.658832,6.939792,49.50,-206.500,-207.500,1.000712,1.000000,1.000000,50,223.000000,335.571429,300.428571,0.0,0.0,0.0
6,LKDS-00001,127.537884,8.934058,286.668803,7.969219,49.50,-206.500,-207.500,1.000712,1.000000,1.000000,50,237.000000,265.433962,385.037736,0.0,0.0,0.0
7,LKDS-00001,64.623236,81.495103,333.800339,9.126302,49.50,-206.500,-207.500,1.000712,1.000000,1.000000,50,284.097990,337.994975,322.123116,0.0,0.0,0.0
8,LKDS-00003,44.500084,136.000049,48.596968,8.486275,45.60,-5.000,-196.000,0.998989,1.000000,1.000000,81,3.000000,222.000000,321.500000,0.0,0.0,0.0
9,LKDS-00003,-2.693002,135.049554,61.084333,10.499243,45.60,-5.000,-196.000,0.998989,1.000000,1.000000,81,15.500000,221.049505,274.306931,0.0,0.0,0.0


In [50]:
pred

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,LKDS-00001,2.000000,252.500000,317.500000,6.873059
1,LKDS-00001,68.000000,303.500000,173.500000,6.873059
2,LKDS-00001,81.000000,326.000000,184.000000,6.203505
3,LKDS-00001,180.573916,205.854796,180.570959,17.981062
4,LKDS-00001,222.427403,293.580777,376.321063,9.774699
5,LKDS-00001,223.000000,335.571429,300.428571,6.939792
6,LKDS-00001,237.000000,265.433962,385.037736,7.969219
7,LKDS-00001,284.097990,337.994975,322.123116,9.126302
8,LKDS-00003,3.000000,222.000000,321.500000,8.486275
9,LKDS-00003,15.500000,221.049505,274.306931,10.499243


In [4]:
patient_id = patients[0].split('/')[-1][:-4]

In [19]:
for idx,row in df.iterrows():
    if row['seriesuid'] == patient_id:
        origin = np.array([row['origin1'],row['origin2'],row['origin3']])
        new_spacing = np.array([row['new_spacing1'],row['new_spacing2'],row['new_spacing3']])

In [55]:
def fine(props, origin, spacing):
    patient_id_ = []
    Centroid3_ = []
    Centroid1_ = []
    Centroid2_ = []
    EquivDiameter_ = []
    var = []
    for i in tqdm(range(len(props))):
        patient_id_ = []
        if props[i]['EquivDiameter'] < 30:
            if props[i]['EquivDiameter'] > 5:
                x=props[i]['Centroid'][0]
                y=props[i]['Centroid'][1]
                z=props[i]['Centroid'][2]
                voxel_coordinates = [z,y,x]
                
                
                world = voxel_2_world(voxel_coordinates, origin, spacing)
                
                patient_id_.append(int(x))
                patient_id_.append(int(y))
                patient_id_.append(int(z))
                patient_id_.append(int(props[i]['EquivDiameter']))
                
                var.append(patient_id_)
    return var

In [58]:
from skimage import data,filters
from sklearn import preprocessing 

min_max_scaler = preprocessing.MinMaxScaler()

def cc(img_):
    var1 = []
    if True:
        img_, origin, spacing = load_itk('/Volumes/solo/ali/Data/train/train_subset00/LKDS-00001.mhd')
        
        #aa = np.load(_)
        #patient_id = _.split('/')[-1][:-4]
        img_ = np.squeeze(img_) 
        img_[img_ >= 400] = 400
        img_[img_ <= -1000] = 0
        #thresh = filters.threshold_otsu(img_)
        #img_ =(img_ <= thresh)*1.0
        #ball = skimage.morphology.ball(2)
        for i in range(img_.shape[0]):
            img_[i] = min_max_scaler.fit_transform(img_[i])

        #temp = skimage.morphology.black_tophat (img_, np.ones([5,5,5]))            
        #temp = skimage.morphology.binary_closing(img_, np.ones([5,5,5]))
        labels = measure.label(np.squeeze(img_))
        props = regionprops(labels)
        var1 =  fine(props, origin, spacing)
    return var1

In [59]:
cc= cc(img)

 15%|█▍        | 2533/16907 [00:00<00:02, 5167.45it/s]

[510.63441712926249, 117.78059740946338, 4.9936558287073751]
[50.770888746216635, 292.03687058607721, 12.906447766669723]
[510.60973684210524, 392.78289473684208, 4.9705263157894741]
[447.50167943033722, 313.459626494693, 10.008195620045681]
[163.23560460652592, 437.20777351247602, 10.7989443378119]
[328.15564424173317, 438.28905359179021, 12.141391106043329]
[407.38194444444446, 356.86111111111109, 9.5763888888888893]
[510.75514403292181, 121.06687242798354, 12.5]
[510.75473684210527, 392.72947368421052, 12.494736842105263]
[93.578947368421055, 423.70394736842104, 21.888157894736842]
[509.972706595906, 396.71948445792265, 21.139310083396513]
[509.96692571841675, 119.94108078799928, 21.116031086210011]
[375.38376383763836, 430.58671586715866, 33.649446494464947]
[461.8901098901099, 336.02197802197804, 21.681318681318682]
[207.93449781659388, 321.70742358078604, 22.729257641921397]
[252.63258785942492, 199.55271565495207, 23.664536741214057]
[185.22618212431598, 276.96688648800335, 47.9

 33%|███▎      | 5613/16907 [00:00<00:01, 7715.57it/s]

[208.01219512195121, 442.0609756097561, 71.024390243902445]
[333.23163841807911, 437.44632768361583, 72.734463276836152]
[167.71573604060913, 438.0558375634518, 72.89340101522842]
[211.78636363636363, 350.43636363636364, 79.059090909090912]
[349.95736160188454, 329.37809187279152, 88.804240282685512]
[140.82498772704957, 335.94378988708888, 93.555964653902805]
[278.59302325581393, 202.12790697674419, 83.0]
[286.94202898550725, 356.68115942028987, 82.898550724637687]
[256.72668810289389, 149.12540192926045, 85.524115755627008]
[171.6683848797251, 438.18384879725085, 91.895189003436428]
[311.70097087378639, 439.76116504854372, 89.132038834951459]
[269.59748427672957, 171.9748427672956, 91.371069182389931]
[256.25104699860401, 148.04606793857607, 130.78222429036762]
[354.5240473061761, 335.58265440210249, 105.87963206307491]
[144.58316485940085, 340.61146848005882, 111.04521227715493]
[413.94715447154471, 265.61382113821139, 102.96747967479675]
[385.01428571428573, 204.80000000000001, 101

 41%|████      | 6860/16907 [00:00<00:01, 8711.87it/s]

[509.0, 109.5, 121.0]
[511.0, 119.5, 121.0]
[511.0, 391.5, 121.0]
[509.0, 401.5, 121.0]
[324.33127889060091, 438.00770416024653, 127.20801232665639]
[11.0, 89.5, 122.0]
[503.0, 94.5, 122.5]
[503.0, 416.5, 122.5]
[11.0, 421.5, 122.0]
[0.0, 119.5, 123.0]
[7.0, 97.755725190839698, 123.49872773536896]
[0.0, 391.5, 123.0]
[7.0, 414.5, 123.5]
[511.0, 119.5, 126.0]
[510.99254349627176, 390.81524440762223, 126.00911350455675]
[468.11818181818182, 280.90909090909093, 142.18181818181819]
[12.0, 87.5, 125.0]
[12.0, 423.5, 125.0]
[176.12382445141066, 438.06583072100312, 133.24294670846396]
[7.0, 96.5, 127.0]
[7.0, 414.5, 127.0]
[10.0, 91.0, 128.0]
[377.32919254658384, 162.2111801242236, 132.13664596273293]
[10.0, 420.0, 128.0]
[0.0, 119.5, 130.5]
[7.0, 96.5, 130.0]
[7.0, 414.5, 130.0]
[14.0, 85.0, 131.0]
[14.0, 426.0, 131.0]
[10.497252747252746, 90.508241758241752, 132.0]
[510.67796610169489, 117.56779661016949, 132.66101694915255]
[10.495867768595041, 420.74380165289256, 132.0]
[26.98901098901098

 63%|██████▎   | 10597/16907 [00:00<00:00, 11356.74it/s]

[121.47746671218846, 333.25460908159783, 208.19725162171389]
[488.06474820143887, 239.5179856115108, 183.97841726618705]
[181.8641975308642, 290.22222222222223, 178.75308641975309]
[25.108695652173914, 246.12422360248448, 197.04037267080744]
[334.37837837837839, 109.31081081081081, 183.81081081081081]
[327.05736137667304, 437.31357552581261, 190.8374760994264]
[20.684410646387832, 229.19391634980988, 195.91634980988593]
[483.36944745395448, 234.08667388949078, 213.48320693391116]
[388.82362585210115, 336.12693071015616, 224.4284235050479]
[497.65068493150687, 270.42465753424659, 200.5958904109589]
[127.218633208756, 344.85735896066916, 230.11025093432994]
[485.48863636363637, 340.46590909090907, 206.93939393939394]
[497.25757575757575, 278.54545454545456, 201.87878787878788]
[494.62619047619046, 270.74087301587304, 229.23134920634919]
[16.368421052631579, 295.0263157894737, 205.15789473684211]
[342.62812500000001, 434.52812499999999, 218.55703124999999]
[19.783505154639176, 237.7525773

100%|██████████| 16907/16907 [00:00<00:00, 18992.34it/s]

[88.728395061728392, 173.32098765432099, 255.32098765432099]
[491.6753246753247, 229.80519480519482, 261.87012987012986]
[491.82269503546098, 276.6241134751773, 261.75177304964541]
[29.596551724137932, 300.58275862068967, 265.49310344827586]
[314.964234620887, 438.35336194563661, 270.35336194563661]
[31.255813953488371, 197.89922480620154, 263.48062015503876]
[493.12676056338029, 259.92605633802816, 264.47183098591552]
[26.892086330935253, 264.79856115107913, 264.61151079136692]
[216.45935727788279, 441.42722117202271, 271.27788279773159]
[34.074074074074076, 286.04938271604937, 266.11111111111109]
[487.91891891891891, 307.90540540540542, 264.40540540540542]
[435.58666666666664, 188.97333333333333, 268.13333333333333]
[98.822784810126578, 424.1645569620253, 266.02531645569621]
[26.486842105263158, 277.60526315789474, 268.69736842105266]
[444.08823529411762, 411.35294117647061, 273.58823529411762]
[79.54069767441861, 417.97093023255815, 277.18604651162792]


In [60]:
sorted(cc)

[[4, 117, 510, 19],
 [4, 392, 510, 19],
 [9, 356, 407, 6],
 [10, 313, 447, 24],
 [10, 437, 163, 15],
 [12, 121, 510, 12],
 [12, 292, 50, 27],
 [12, 392, 510, 12],
 [12, 438, 328, 14],
 [21, 119, 509, 21],
 [21, 336, 461, 5],
 [21, 396, 509, 21],
 [21, 423, 93, 6],
 [22, 321, 207, 7],
 [23, 199, 252, 8],
 [28, 121, 511, 9],
 [28, 391, 511, 9],
 [32, 116, 509, 14],
 [33, 430, 375, 8],
 [35, 299, 442, 15],
 [36, 394, 0, 19],
 [38, 340, 209, 5],
 [38, 435, 348, 7],
 [40, 433, 136, 11],
 [43, 338, 288, 5],
 [45, 101, 5, 12],
 [45, 410, 5, 12],
 [47, 276, 185, 23],
 [47, 280, 442, 5],
 [54, 307, 158, 22],
 [55, 309, 343, 22],
 [59, 109, 509, 7],
 [59, 401, 509, 7],
 [59, 422, 94, 6],
 [61, 354, 211, 5],
 [62, 341, 285, 7],
 [65, 216, 227, 12],
 [71, 322, 361, 18],
 [71, 442, 208, 5],
 [72, 326, 155, 20],
 [72, 437, 333, 6],
 [72, 438, 167, 7],
 [79, 350, 211, 7],
 [82, 356, 286, 5],
 [83, 202, 278, 5],
 [85, 149, 256, 8],
 [88, 329, 349, 20],
 [89, 439, 311, 9],
 [91, 171, 269, 6],
 [91, 438

In [ ]:
img = np.squeeze(img)
#temp = skimage.morphology.binary_opening(img, np.ones([5,5,5]))
labels = measure.label(np.squeeze(temp))
props = regionprops(labels)

In [ ]:
len(props)

In [18]:
os.listdir('/Volumes/solo/ali/Data/train/train_subset00/LKDS-00001.mhd')

['LKDS-00001.mhd',
 'LKDS-00001.zraw',
 'LKDS-00003.mhd',
 'LKDS-00003.zraw',
 'LKDS-00004.mhd',
 'LKDS-00004.zraw',
 'LKDS-00005.mhd',
 'LKDS-00005.zraw',
 'LKDS-00007.mhd',
 'LKDS-00007.zraw',
 'LKDS-00011.mhd',
 'LKDS-00011.zraw',
 'LKDS-00013.mhd',
 'LKDS-00013.zraw',
 'LKDS-00015.mhd',
 'LKDS-00015.zraw',
 'LKDS-00016.mhd',
 'LKDS-00016.zraw',
 'LKDS-00019.mhd',
 'LKDS-00019.zraw',
 'LKDS-00020.mhd',
 'LKDS-00020.zraw',
 'LKDS-00021.mhd',
 'LKDS-00021.zraw',
 'LKDS-00023.mhd',
 'LKDS-00023.zraw',
 'LKDS-00025.mhd',
 'LKDS-00025.zraw',
 'LKDS-00026.mhd',
 'LKDS-00026.zraw',
 'LKDS-00028.mhd',
 'LKDS-00028.zraw',
 'LKDS-00029.mhd',
 'LKDS-00029.zraw',
 'LKDS-00030.mhd',
 'LKDS-00030.zraw',
 'LKDS-00034.mhd',
 'LKDS-00034.zraw',
 'LKDS-00035.mhd',
 'LKDS-00035.zraw',
 'LKDS-00036.mhd',
 'LKDS-00036.zraw',
 'LKDS-00038.mhd',
 'LKDS-00038.zraw',
 'LKDS-00039.mhd',
 'LKDS-00039.zraw',
 'LKDS-00040.mhd',
 'LKDS-00040.zraw',
 'LKDS-00041.mhd',
 'LKDS-00041.zraw',
 'LKDS-00042.mhd',
 'LKDS

In [25]:
img.shape

(351, 1, 512, 512)